<a href="https://colab.research.google.com/github/shitote/dist-repo/blob/main/transfer_learning_scaling_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rranfer Learnig Scaling up (🍰👁 Food Vision)

In [ ]:
!nvidia-smi

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [3]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir

## 101 Working with less data

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
unzip_data("101_food_classes_10_percent.zip")

In [5]:
train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"

In [ ]:
# image classes
walk_through_dir("101_food_classes_10_percent")

In [ ]:
# Se up data inputs
import tensorflow as tf
IMG_SIZE = (224, 224)
train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                               label_mode="categorical",
                                                                               image_size=IMG_SIZE)
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode='categorical',
                                                                image_size=IMG_SIZE,
                                                                shuffle=False)

## Train the model.
steps to take.
* Create a modelCheckpoint callback
* Create a data augmentation.
* Build a headlsess functional api model.
* compile the model.
* Feature extraxt


In [8]:
checkpoint_path = "101_classes_10_percent_data_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True,
                                                         monitor="val_accuracy",
                                                         save_best_only=True)

In [9]:
# Create data augmentation layer
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

data_augmentaion = Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2)
], name="dara_augmentation")


In [ ]:
# set up the base model.
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Setup model architecture with trainabe top layers
inputs = layers.Input(shape=(224, 224, 3), name='input_layer')
x = data_augmentaion(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D(name='gloabal_avg_pool_layer')(x)
outputs = layers.Dense(len(train_data_all_10_percent.class_names), activation="softmax", name="output_layer")(x)
model = tf.keras.Model(inputs, outputs)



In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

# Fit the model
history_all_classes_10_percent = model.fit(train_data_all_10_percent,
                                           epochs=5,
                                           validation_data=test_data,
                                           validation_steps=int(0.15 * len(test_data)),
                                           callbacks=[checkpoint_callback])

In [ ]:
fine_tuning_results = model.evaluate(test_data)
fine_tuning_results

In [ ]:
plot_loss_curves(history_all_classes_10_percent)

## FIne tuning

In [14]:
# Unfreeze all the layers in the base model to be trainable
base_model.trainable = True

# Refreeze every layer excep the last 5
for layer in base_model.layers[:-5]:
  layer.trainable = False

In [15]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.0001),
              metrics=['accuracy'])

In [ ]:
for layer in model.layers:
  print(layer.name, layer.trainable)

In [ ]:
for layer_number, layer in enumerate(model.layers[2].layers):
  print(layer_number, layer.name, layer.trainable)


In [ ]:
# Fien tune for 5 more epoches
fine_tune_epochs = 10

# Fine-tune the model.
history_all_classes_10_percent_fine_tune = model.fit(train_data_all_10_percent,
                                                     epochs=fine_tune_epochs,
                                                     validation_data=test_data,
                                                     validation_steps=int(0.15 * len(test_data)),
                                                     initial_epoch=history_all_classes_10_percent.epoch[-1])

In [ ]:
history_all_classes_10_percent_fine_tune_results = model.evaluate(test_data)

In [ ]:
compare_historys(history_all_classes_10_percent,
    history_all_classes_10_percent_fine_tune)

## Saving and loading the model
To use the model is an external application, we'll need to save it export it somewhare else.


In [21]:
model.save("drive/MyDrive/tensorflow/101_food_classes_10_percent_saved_big_dog_model")

In [22]:
import tensorflow as tf
loaded_model = tf.keras.models.load_model("drive/MyDrive/tensorflow/101_food_classes_10_percent_saved_big_dog_model")

In [ ]:
loaded_model_results = loaded_model.evaluate(test_data)

In [ ]:
history_all_classes_10_percent_fine_tune_results

## Evaluating the performance of the model across all different classes

In [ ]:
import tensorflow as tf

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/06_101_food_class_10_percent_saved_big_dog_model.zip

In [26]:
unzip_data("/content/06_101_food_class_10_percent_saved_big_dog_model.zip")

In [ ]:
model = tf.keras.models.load_model("/content/06_101_food_class_10_percent_saved_big_dog_model")

In [ ]:
# Evaluate the loaded model on test data
results_downloaded_model = model.evaluate(test_data)


In [ ]:
results_downloaded_model

## Making predictions.


In [ ]:
# make predictions with the model.
preds_probs = model.predict(test_data, verbose=1)

In [ ]:
# How many predictions are there
len(preds_probs), preds_probs.shape

In [ ]:
preds_probs[:10]

In [ ]:
len(preds_probs[0]), sum(preds_probs[0]), preds_probs[0]

The model outputs a prediction probability array(with N number of variables, wher N is the number of classes) for each sample passed to the prediction method

In [ ]:
# get one prediction probability per class.
print(f"Number of prediction probs sample 0: {len(preds_probs[0])}")
print(f"Prediction prob for sample 0: {preds_probs[0]}")
print(f"The highest prediction prob for the results in the prediction array: {preds_probs[0].argmax()}")

In [ ]:
print(len(test_data) * 32)

In [ ]:
pred_classes = preds_probs.argmax(axis=1)
pred_classes[:10]

COmapre the prediction to the original sample

In [ ]:
# Unravle test data batchDataset
y_labels = []
for images, labels in test_data.unbatch():
  y_labels.append(labels.numpy().argmax())  # Find the index value where one occuers in the batch


y_labels[:10]

## Evaluate the prediction for the model.
Check the models prediction are the same order as the test labels using the accuracy score.

In [38]:
results_downloaded_model

[1.8027204275131226, 0.6077623963356018]

In [39]:
from sklearn.metrics import accuracy_score
sklearn_accuracy = accuracy_score(y_true = y_labels,
                                  y_pred=pred_classes)
sklearn_accuracy

0.6077623762376237

In [40]:
import numpy as np
np.isclose(results_downloaded_model[1], sklearn_accuracy)

True

## Make confusion matrix

In [41]:
from helper_functions import make_confusion_matrix

class_names = test_data.class_names

In [42]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

# Our function needs a different name to sklearn's plot_confusion_matrix
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False):
  """Makes a labelled confusion matrix comparing predictions and ground truth labels.

  If classes is passed, confusion matrix will be labelled, if not, integer class values
  will be used.

  Args:
    y_true: Array of truth labels (must be same shape as y_pred).
    y_pred: Array of predicted labels (must be same shape as y_true).
    classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
    figsize: Size of output figure (default=(10, 10)).
    text_size: Size of output figure text (default=15).
    norm: normalize values or not (default=False).
    savefig: save confusion matrix to file (default=False).

  Returns:
    A labelled confusion matrix plot comparing y_true and y_pred.

  Example usage:
    make_confusion_matrix(y_true=test_labels, # ground truth test labels
                          y_pred=y_preds, # predicted labels
                          classes=class_names, # array of class label names
                          figsize=(15, 15),
                          text_size=10)
  """
  # Create the confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)

  # Are there a list of classes?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])

  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes),
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)

  # Make x-axis labels appear on bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  ## plot x labels vertically
  plt.xticks(rotation=70, fontsize=text_size)
  plt.yticks(fontsize=text_size)

  # Set the threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if norm:
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else:
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

  # Save the figure to the current working directory
  if savefig:
    fig.savefig("confusion_matrix.png")


In [ ]:
make_confusion_matrix(y_true=y_labels,
                      y_pred=pred_classes,
                      classes=class_names,
                      figsize=(100, 100),
                      text_size=20)

### Classification report

THis is a function form sklearn library

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true= y_labels,
                            y_pred=pred_classes))

In [45]:
classification_report_dict = classification_report(y_labels, pred_classes, output_dict=True)

In [ ]:
# plot all of empty dictionary
class_f1_scores = {}
for k, v in classification_report_dict.items():
  if k == "accuracy":  # stop once you get to accuracy
    break
  else:
    class_f1_scores[class_names[int(k)]] = v["f1-score"]
class_f1_scores

In [ ]:
import pandas as pd
f1_score = pd.DataFrame({"class_name": list(class_f1_scores.keys()),
                         "f1-score": list(class_f1_scores.values())}).sort_values("f1-score", ascending=False)
f1_score

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(12, 25))
scores = ax.barh(range(len(f1_score)), f1_score["f1-score"].values) # get f1_score values
ax.set_yticks(range(len(f1_score)))
ax.set_yticklabels(f1_score["class_name"])
ax.set_xlabel("F1-score")
ax.set_title("F1-score for 101 Different food classes")
ax.invert_yaxis();

## isualizing prediction on custom images.
Now this is the real test, how the modal go on food images not even on the test dataset

Create a function to preprocess images, specifically it will need to:
* Read in a target image filepath using tf.io.read_file()
* Turn the images into a Tensor using tf.io.decode_image()
* resize  the image tensor to be the same as the image the  model trained on using tf.image.resize()
* scale the image to get all of the pixel values between 0 and 1

In [60]:
def load_and_pre_image(filename, img_shape=224, scale=True):
  """
  Reads in an image from filename, turn it into a tensor and reshape into
  specific shape Iimg_shape, img_shape, color_channels.
  Args:
    Filename (str): path to target image.
    image_shape (int): height/width dimention fo target image size.
    scale (bool): sccale pixel values from 0 - 255 to 0-1 or not.

  Returns:
    Image tesor of shape (img_shape, img_shape, 3)
  """
  # Read in the image.
  img = tf.io.read_file(filename)

  # Decode the image
  img = tf.io.decode_image(img, channels=3)

  # REsize the image'
  img = tf.image.resize(img, [img_shape, img_shape])

  # if Scale ? yes/no
  if scale:
    # rescale the image (get all values between 0 and 1)
    return img/255.
  else: return img



* Load a few random images from the test dataset
* Make predicrion on the loaded image.
* plot the original image along with the model's predictions, prediction probability and truth label


### Make preds on a serices of random images.

In [ ]:
import os
import random

plt.figure(figsize=(17, 10))
for i in range(3):
  # Choose a random image form a random class
  class_name = random.choice(class_names)
  filename = random.choice(os.listdir(test_dir + "/" + class_name))
  filepath = test_dir + class_name + '/' + filename

  # Load the images and make predictions
  img = load_and_pre_image(filepath, scale=False)
  img_expanded = tf.expand_dims(img, axis=0)
  # print(img_expanded.shape)
  pred_prob = model.predict(img_expanded)  # an array of prediction probabilities.
  pred_class = class_names[pred_prob.argmax()]  # get highest prediction prob
  # print(pred_prob, pred_class)

  # Plot the images(s)
  plt.subplot(1, 3, i+1)
  # print(img)
  plt.imshow(img/225.1)
  if class_name == pred_class:
    title_color = 'g'
  else:
    title_color = 'r'
  plt.title(f"actual: {class_name}, pred: {pred_class}, prob: {pred_prob.max():.2f}", c=title_color)
  plt.axis(False)

In [ ]:
import random
random.choice(class_names)